In [ ]:
import os
import shutil

if os.path.isfile("masked-face-dataset.zip"):
  os.remove('masked-face-dataset.zip')
  
if os.path.exists("masked-face-dataset"):
  shutil.rmtree('masked-face-dataset')

!wget https://storage.googleapis.com/ib-ml-bucket/masked-face-dataset.zip
!unzip 'masked-face-dataset.zip'

In [ ]:
import os
import shutil

test_data_filename = 'masked-face-test-v2';

if os.path.isfile(f'{test_data_filename}.zip'):
  os.remove(f'{test_data_filename}.zip')

if os.path.exists(f'{test_data_filename}'):
  shutil.rmtree(f'{test_data_filename}')

!wget https://storage.googleapis.com/ib-ml-bucket/masked-face-test-v2.zip
!unzip masked-face-test-v2.zip

In [ ]:
# import tensorflow as tf
# try:
#   resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#   tf.config.experimental_connect_to_cluster(resolver)
#   tf.tpu.experimental.initialize_tpu_system(resolver)
#   print("All devices: ", tf.config.list_logical_devices('TPU'))
#   strategy = tf.distribute.experimental.TPUStrategy(resolver)
# except ValueError:
#   strategy = tf.distribute.get_strategy() 


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import pathlib
import numpy as np

dataset_root_dir = 'masked-face-dataset'
test_case = f'{dataset_root_dir}/cloth/full_face'

train_dir = f'{test_case}/train'
validation_dir = f'{test_case}/validation'

img_width = 256
img_height = 256
batch_size_train = 8
batch_size_val = 8

# Get the class names
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted(item.name for item in data_dir.glob('*')))

# Preprocessing the data (scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1.0/255)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Import data from directories and turn in into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(img_width, img_height),
                                               class_mode="categorical",
                                               batch_size=batch_size_train,
                                               shuffle=True)

validation_data = validation_datagen.flow_from_directory(directory=validation_dir,
                                                         target_size=(img_width, img_height),
                                                         class_mode="categorical",
                                                         batch_size=batch_size_train,
                                                         shuffle=True)

train_data_gen_augmented = ImageDataGenerator(rescale=1/255.,
                                              rotation_range=10,
                                              zoom_range=.2,
                                              # width_shift_range=.2,
                                              # height_shift_range=.2,
                                              # fill_mode="constant",
                                              horizontal_flip=True,
                                              # vertical_flip=True,
                                              # shear_range=0.1,
                                              preprocessing_function=preprocess_input
                                              )

# Better result with batch_size = 8
train_data_augmented = train_data_gen_augmented.flow_from_directory(directory=train_dir,
                                                                    target_size=(img_width, img_height),
                                                                    class_mode="categorical",
                                                                    batch_size=batch_size_val)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten, Dropout
from keras.models import Model

def model_VGG16():
    # Load the VGG16 model
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers
    for layer in vgg16.layers:
        layer.trainable = False

    # Build new classifier
    x = vgg16.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.1)(x)
    x = Dense(len(class_names), activation='softmax')(x)

    # Create new model
    return Model(inputs=vgg16.input, outputs=x)

In [ ]:
from keras.optimizers import Adam

model = model_VGG16()
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=1e-4),
              metrics=["accuracy"])
model_summary = model.summary()

In [ ]:
import time

start_history = time.perf_counter()

# model = load_model(filepath='./model')
# steps_per_epoch = round((len(train_data_augmented) / batch_size_train))
steps_per_epoch = 64

history = model.fit(train_data_augmented,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_data,
                    validation_steps=batch_size_val)

elapsed_history = time.perf_counter() - start_history

In [ ]:
train_accuracy = model.evaluate(train_data_augmented)
print(f'Training Loss : {train_accuracy[0]}')
print(f'Training Accuracy : {round(train_accuracy[1] * 100, 2)}%\n')

val_accuracy = model.evaluate(validation_data)
print(f'Validation Loss : {val_accuracy[0]}')
print(f'Validation Accuracy : {round(val_accuracy[1] * 100, 2)}%')

In [ ]:
from keras.utils import load_img
import matplotlib.image as mpimpg
import matplotlib.pyplot as plt
import glob

faceimagedir = test_data_filename

images = []
images_paths = []
for img_path in glob.glob(faceimagedir + '/*.jpg'):
    images.append(mpimpg.imread(img_path))
    images_paths.append(img_path)
    
plt.figure(figsize=(14, 8))
columns = 6

for i, image in enumerate(images):
    img = load_img(images_paths[i], target_size=(img_width, img_height))

    image_to_predict = np.expand_dims(img, axis=0)
    predictions = model.predict(image_to_predict)
    classes = np.argmax(predictions, axis = 1)
    
    rows = int(len(images) / columns + 1)
    plt.subplot(rows, columns, i + 1)
    # plt.text(4, 58, class_names[classes][0], bbox={'facecolor': 'white', 'pad': 3})
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(class_names[classes][0])
    # plt.xlabel(predictions)
    plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img, label = train_data_augmented.next()

plt.figure(figsize=(15, 8))

# generate batch of images
for i in range(0, 5):
    # fig = plt.figure()
    plt.subplot(1, 5, 1 + i)
    # ax.set_title()
    plt.imshow(img[i])

plt.show()

In [ ]:
## Create directory to save the result
test_time = time.strftime('%Y%m%d_%H%M')
result_dir = f'results/{test_case}/{test_time}'
if (os.path.exists(result_dir) == False):
        os.makedirs(result_dir)
        
## Write History Time and Evaluate into a txt file
f = open(f'{result_dir}/result_data.txt', 'a')

f.write('- fit time \t: %.3f seconds' % elapsed_history)
f.write('\n\n')

f.write(f'- image_width \t: {img_width}')
f.write(f'- image_height \t: {img_height}')
f.write('\n\n')

f.write(f'- steps per epoch \t: {steps_per_epoch}')
f.write(f'- train batch size \t: {batch_size_train}')
f.write(f'- val batch size \t: {batch_size_val}')
f.write('\n\n')

f.write('- train_loss \t: %.5f\n' % train_accuracy[0])
f.write("- train_acc \t: {:.2%}".format(train_accuracy[1]))
f.write('\n\n')

f.write('- val_loss \t: %.5f\n' % val_accuracy[0])
f.write("- val_acc \t: {:.2%}".format(val_accuracy[1]))
f.close()


# Save the model
str_percentage = str(round(val_accuracy[1] * 100, 2))
# model_filename = "model_" + test_case + "_" + str_percentage.replace('.', '_')
model_filename = "model"
model_json = model.to_json()
with open(f'{result_dir}/{model_filename}.json', "w") as json_file:
    json_file.write(f'{result_dir}/{model_json}')

# model.save_weights(f'{result_dir}/{model_filename}_weight.h5')
model.save(f'{result_dir}/{model_filename}', save_format='h5')

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
    """
    Plots the loss curves from the training history.
    """
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    # plt.show()

    return plt

def plot_accuracy_curves(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    # plt.show()

    return plt

In [ ]:
loss_plot = plot_loss_curves(history)
loss_plot.savefig(f'{result_dir}/loss_graph.svg')

In [ ]:
acc_plot = plot_accuracy_curves(history)
acc_plot.savefig(f'{result_dir}/acc_graph.svg')

In [ ]:
# from keras.utils import load_img
# img = load_img('masked-face-test/ibnu_01.jpg', target_size=(img_width, img_height))

# image_to_predict = np.expand_dims(img, axis=0)
# predictions = model.predict(image_to_predict)
# classes = np.argmax(predictions, axis = 1)
# print(predictions)